In [6]:
# imports
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    TimeDistributed,
    LSTM,
    Dense,
    Flatten,
    Conv1D,
    Reshape,
    Dropout,
    Concatenate,
    GaussianNoise,
)
from neptune.integrations.tensorflow_keras import NeptuneCallback

import constants
from solarprophet import TabularTest
import utils

## import data, model, and initial feature importance


In [2]:
args = dict(
    n_steps_in=1,
    n_steps_out=12,
    selected_groups=None,
    selected_responses=["Delta CSI GHI"],
    scaler_type="minmax",
    data_path=constants.JOINT_DATA_H5_PATH,
    ## Saving and cache parameters (optional)
    model_save_path="results",
    datetimes_cache_path=constants.DATETIME_VALUES_PATH,
    window_cache_path=os.path.join("data", "windows_cache.h5"),
    all_past_features=constants.PAST_FEATURES,
    all_future_features=constants.FUTURE_FEATURES,
    all_scalar_responses=constants.SCALAR_RESPONSES,
    all_relative_responses=constants.RELATIVE_RESPONSES,
    # Model (optional)
    model=None,
    model_name="NoiseModel",
    # Optimizer and training tuning parameters (optional)
    scale_responses=True,
    epochs=2000,
    shuffle_training_order=False,
    batch_size=2**14,
    loss="mae",
    optimizer=None,
    callbacks=[],
    early_stopping=True,
    stopping_patience=1000,
    stopping_min_delta=1e-4,
    metrics=[],  # todo possibly replace
    fit_verbose=1,
    # Utility Parameters (optional)
    data_cols=constants.DATA_COLS,
    feature_groups=constants.FEATURE_GROUPS,
    scalar_response=constants.SCALAR_RESPONSES,
    relative_response=constants.RELATIVE_RESPONSES,
    seed=42,
    n_job_workers=1,
    # Neptune Parameters (optional)
    neptune_log=True,
    neptune_run_name="",
)
train_validate_dates = ["2019-09-27", "2020-09-27", "2021-09-27"]
end_dates = ["2020-09-27", "2021-09-27", "2022-09-27"]

In [ ]:
for i in range(3):
    tt = TabularTest(
        selected_features=constants.PAST_FEATURES,
        noise_model=True,
        tags=["NoiseModel-True", "Top 10 Features"],
        learning_rate=tf.optimizers.schedules.CosineDecayRestarts(
            1e-2,  # initial learning rate
            600,  # number of steps to initially decay over
            t_mul=3,  # more steps to run over at restart
            m_mul=0.5,  # restart maximum learning rate multiplier
            alpha=0,
        ),
        **args
    )

    tt.import_preprocess_cached_windows(train_validate_dates[i], end_dates[i], True)
    tt.compile_model()
    tt.fit_model()
    tt.final_error_metrics(cv=i + 1)
    tt.save_and_quit(cv=i + 1)

In [ ]:
for i in range(3):
    tt = TabularTest(
        selected_features=[
            "BRBG Total Cloud Cover [%]",
            "CSI GHI",
            "dni t-4",
            "940nm POM-01 Photometer [uA]",
            "cs_dev_mean t-10 dni",
            "675nm POM-01 Photometer [uA]",
            "CSI DNI",
            "elevation",
            "cs_dev t dni",
            "dni t-9",
        ],
        noise_model=True,
        tags=["NoiseModel-True", "Top 10 Features"],
        learning_rate=tf.optimizers.schedules.CosineDecayRestarts(
            1e-2,  # initial learning rate
            600,  # number of steps to initially decay over
            t_mul=3,  # more steps to run over at restart
            m_mul=0.5,  # restart maximum learning rate multiplier
            alpha=0,
        ),
        **args
    )

    tt.import_preprocess_cached_windows(train_validate_dates[i], end_dates[i], True)
    tt.compile_model()
    tt.fit_model()
    tt.final_error_metrics(cv=i + 1)
    tt.save_and_quit(cv=i + 1)

In [ ]:
for i in range(3):
    tt = TabularTest(
        selected_features=constants.PAST_FEATURES,
        noise_model=False,
        tags=["NoiseModel-False", "All Features"],
        learning_rate=tf.optimizers.schedules.CosineDecayRestarts(
            1e-2,  # initial learning rate
            600,  # number of steps to initially decay over
            t_mul=3,  # more steps to run over at restart
            m_mul=0.5,  # restart maximum learning rate multiplier
            alpha=0,
        ),
        **args
    )

    tt.import_preprocess_cached_windows(train_validate_dates[i], end_dates[i], True)
    tt.compile_model()
    tt.fit_model()
    tt.final_error_metrics(cv=i + 1)
    tt.save_and_quit(cv=i + 1)

In [ ]:
for i in range(3):
    tt = TabularTest(
        selected_features=[
            "BRBG Total Cloud Cover [%]",
            "CSI GHI",
            "dni t-4",
            "940nm POM-01 Photometer [uA]",
            "cs_dev_mean t-10 dni",
            "675nm POM-01 Photometer [uA]",
            "CSI DNI",
            "elevation",
            "cs_dev t dni",
            "dni t-9",
        ],
        noise_model=False,
        tags=["NoiseModel-False", "Top 10 Features"],
        learning_rate=tf.optimizers.schedules.CosineDecayRestarts(
            1e-2,  # initial learning rate
            600,  # number of steps to initially decay over
            t_mul=3,  # more steps to run over at restart
            m_mul=0.5,  # restart maximum learning rate multiplier
            alpha=0,
        ),
        **args
    )

    tt.import_preprocess_cached_windows(train_validate_dates[i], end_dates[i], True)
    tt.compile_model()
    # tt.fit_model()
    tt.callbacks.extend(
        [
            EarlyStopping(
                monitor="val_loss",
                min_delta=tt.stopping_min_delta,
                patience=tt.stopping_patience,
                verbose=tt.fit_verbose,
                mode="min",
                restore_best_weights=True,
            ),
            NeptuneCallback(run=tt.run),
        ]
    )
    tt.train_inputs = tt.train_past_features
    tt.validation_inputs = tt.validate_past_features
    history = tt.model.fit(
        x=tt.train_inputs,  # TODO save Model Weights in Neptune
        y=tt.y_train_true,
        validation_data=(tt.validation_inputs, tt.y_validate_true),
        epochs=tt.epochs,
        batch_size=tt.batch_size,
        shuffle=tt.shuffle_training_order,
        callbacks=tt.callbacks,
        workers=tt.n_job_workers,
        verbose=tt.fit_verbose,
    )
    tt.final_error_metrics(cv=i + 1)
    tt.save_and_quit(cv=i + 1)

https://app.neptune.ai/HorizonPSE/SolarProphet-II/e/SOL-311
====== Reading windows from cache ======
past_features.shape: (78930, 1, 134)

Done


Relative Response
Train windows: 30515

self.train_dates.shape=(30515, 13)

self.train_past_features.shape=(30515, 1, 10)

self.train_future_features.shape=(30515, 12, 0)

self.train_scalar_responses.shape=(30515, 12, 0)

self.train_relative_responses.shape=(30515, 12, 1)

self.train_clear_sky_indexes.shape=(30515, 13)

self.train_clear_sky_irradiances.shape=(30515, 13)
Validate windows: 17218


Model: "NoiseModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 10)]           0         
                                                                 
 PastInputsExpanded (Lambda  (None, 1, 10, 1)          0         
 )                                                               
                                                                 
 time_distributed (TimeDist  (None, 1, 10, 32)         64        
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 1, 320)            0         
 stributed)                                                      
                                                                 
 LSTM (LSTM)                 (None, 16)                21568     
                                                        

2024-02-23 12:47:24.935564: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-23 12:47:25.131783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/2 [==============>...............] - ETA: 1s - loss: 0.4731

...

2024-02-23 12:57:18.234881: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-23 12:57:18.299265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


411/411 [==============================] - 2s 4ms/step
INFO:tensorflow:Assets written to: results/SOL-313_CV3/assets


INFO:tensorflow:Assets written to: results/SOL-313_CV3/assets


Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptun

Exception in thread Exception in thread NeptuneWebhooks:
Traceback (most recent call last):
  File "/Users/jeh/anaconda3/envs/tabsof/lib/python3.8/site-packages/websocket/_socket.py", line 108, in recv
Exception in thread NeptuneWebhooks:
Traceback (most recent call last):
  File "/Users/jeh/anaconda3/envs/tabsof/lib/python3.8/site-packages/websocket/_socket.py", line 108, in recv
Exception in thread NeptuneWebhooks:
Traceback (most recent call last):
  File "/Users/jeh/anaconda3/envs/tabsof/lib/python3.8/site-packages/websocket/_socket.py", line 108, in recv
Exception in thread NeptuneWebhooks:
Traceback (most recent call last):
  File "/Users/jeh/anaconda3/envs/tabsof/lib/python3.8/site-packages/websocket/_socket.py", line 108, in recv
Exception in thread NeptuneWebhooks:
Traceback (most recent call last):
  File "/Users/jeh/anaconda3/envs/tabsof/lib/python3.8/site-packages/websocket/_socket.py", line 108, in recv
Exception in thread NeptuneWebhooks:
Traceback (most recent call last)